In [1]:
from google.colab import files
uploaded = files.upload()  # upload `gpt-project.zip`


Saving training-gpt-2.zip to training-gpt-2.zip


In [2]:
!unzip -o training-gpt-2.zip -d training-gpt-2
%cd training-gpt-2

Archive:  training-gpt-2.zip
   creating: training-gpt-2/training-gpt-2/
  inflating: training-gpt-2/training-gpt-2/.DS_Store  
  inflating: training-gpt-2/__MACOSX/training-gpt-2/._.DS_Store  
  inflating: training-gpt-2/training-gpt-2/train_fine_tune_with_gpu.ipynb  
  inflating: training-gpt-2/__MACOSX/training-gpt-2/._train_fine_tune_with_gpu.ipynb  
  inflating: training-gpt-2/training-gpt-2/custom_model.py  
  inflating: training-gpt-2/__MACOSX/training-gpt-2/._custom_model.py  
  inflating: training-gpt-2/training-gpt-2/train_fine_tune_with_gpu.py  
  inflating: training-gpt-2/__MACOSX/training-gpt-2/._train_fine_tune_with_gpu.py  
  inflating: training-gpt-2/training-gpt-2/model.py  
  inflating: training-gpt-2/__MACOSX/training-gpt-2/._model.py  
  inflating: training-gpt-2/training-gpt-2/train.py  
  inflating: training-gpt-2/__MACOSX/training-gpt-2/._train.py  
  inflating: training-gpt-2/training-gpt-2/configurator.py  
  inflating: training-gpt-2/__MACOSX/training-gpt-2/._

In [11]:
# 1) Uninstall any conflicting versions
!pip uninstall -y torch torchvision torchaudio transformers accelerate trl peft bitsandbytes numpy datasets

# 2) Upgrade pip and reinstall exactly the versions you want
!pip install --upgrade pip && \
 pip install numpy==1.26.4 && \
 pip install torch==2.0.1 torchvision==0.15.2 torchaudio==2.0.2 \
   --index-url https://download.pytorch.org/whl/cu118 && \
 pip install transformers==4.31.0 \
             accelerate==0.21.0 \
             peft==0.4.0 \
             trl==0.4.7 \
             datasets==2.18.0 \
             tqdm==4.66.1 \
             pandas==2.1.0


Found existing installation: torch 2.0.1+cu118
Uninstalling torch-2.0.1+cu118:
  Successfully uninstalled torch-2.0.1+cu118
Found existing installation: torchvision 0.15.2+cu118
Uninstalling torchvision-0.15.2+cu118:
  Successfully uninstalled torchvision-0.15.2+cu118
Found existing installation: torchaudio 2.0.2+cu118
Uninstalling torchaudio-2.0.2+cu118:
  Successfully uninstalled torchaudio-2.0.2+cu118
Found existing installation: transformers 4.31.0
Uninstalling transformers-4.31.0:
  Successfully uninstalled transformers-4.31.0
Found existing installation: accelerate 0.21.0
Uninstalling accelerate-0.21.0:
  Successfully uninstalled accelerate-0.21.0
Found existing installation: trl 0.4.7
Uninstalling trl-0.4.7:
  Successfully uninstalled trl-0.4.7
Found existing installation: peft 0.4.0
Uninstalling peft-0.4.0:
  Successfully uninstalled peft-0.4.0
Found existing installation: bitsandbytes 0.45.5
Uninstalling bitsandbytes-0.45.5:
  Successfully uninstalled bitsandbytes-0.45.5
Found

In [13]:
%cd training-gpt-2

[Errno 2] No such file or directory: 'training-gpt-2'
/content/training-gpt-2/training-gpt-2


In [14]:
!pwd

/content/training-gpt-2/training-gpt-2


In [7]:
!python prepare.py

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
vocab.json: 100% 1.04M/1.04M [00:00<00:00, 2.48MB/s]
merges.txt: 100% 456k/456k [00:00<00:00, 1.07MB/s]
tokenizer_config.json: 100% 26.0/26.0 [00:00<00:00, 180kB/s]
config.json: 100% 665/665 [00:00<00:00, 4.34MB/s]
Tokenizing: 100% 24184/24184 [00:21<00:00, 1101.67it/s]
Tokenizing: 100% 2688/2688 [00:01<00:00, 1515.43it/s]
✅ Preparation complete. Files and tokenizer saved to:
👉 Encoded data: data/Bitext-customer-support-llm-chatbot-training-dataset
👉 Tokenizer: /content/training-gpt-2/training-gpt-2/out


In [8]:
!python train.py --batch_size=4 --compile=False --eval_iters=2 --eval_interval=2 --max_iters=2

Overriding: batch_size = 4
Overriding: compile = False
Overriding: eval_iters = 2
Overriding: eval_interval = 2
Overriding: max_iters = 2
tokens per iteration will be: 163,840
found vocab_size = 50257 (inside data/Bitext-customer-support-llm-chatbot-training-dataset/meta.pkl)
Initializing a new model from scratch
number of parameters: 123.55M
num decayed parameter tensors: 50, with 124,318,464 parameters
num non-decayed parameter tensors: 25, with 19,200 parameters
using fused AdamW: True
step 0: train loss 10.9303, val loss 10.9375
iter 0: loss 10.9358, time 11127.91ms, mfu -100.00%
iter 1: loss 10.9318, time 10548.59ms, mfu -100.00%
step 2: train loss 10.8553, val loss 10.8646
saving checkpoint to out
iter 2: loss 10.8937, time 20001.09ms, mfu -100.00%


### Load and use the model for Inference

In [4]:
import torch
from model import GPT, GPTConfig
from transformers import AutoTokenizer

# 📍 Path to checkpoint
ckpt_path = "out/ckpt.pt"

# 🧠 Load the checkpoint
checkpoint = torch.load(ckpt_path, map_location="cuda" if torch.cuda.is_available() else "cpu")

# 🧱 Reconstruct the model config and model
config = GPTConfig(**checkpoint["model_args"])
model = GPT(config)

# 🛠️ Fix any unwanted prefixes (DDP case)
state_dict = checkpoint["model"]
unwanted_prefix = "_orig_mod."
for k in list(state_dict.keys()):
    if k.startswith(unwanted_prefix):
        state_dict[k[len(unwanted_prefix):]] = state_dict.pop(k)

model.load_state_dict(state_dict)
model.to("cuda" if torch.cuda.is_available() else "cpu")
model.eval()


number of parameters: 123.55M


GPT(
  (transformer): ModuleDict(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-11): 12 x Block(
        (ln_1): LayerNorm()
        (attn): CausalSelfAttention(
          (c_attn): Linear(in_features=768, out_features=2304, bias=False)
          (c_proj): Linear(in_features=768, out_features=768, bias=False)
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
        )
        (ln_2): LayerNorm()
        (mlp): MLP(
          (c_fc): Linear(in_features=768, out_features=3072, bias=False)
          (gelu): GELU(approximate='none')
          (c_proj): Linear(in_features=3072, out_features=768, bias=False)
          (dropout): Dropout(p=0.0, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm()
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [5]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [6]:
def generate(prompt, max_new_tokens=100):
    device = next(model.parameters()).device
    model.eval()
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

    with torch.no_grad():
        output = model.generate(input_ids, max_new_tokens=max_new_tokens)

    return tokenizer.decode(output[0], skip_special_tokens=True)


In [7]:
prompt = "My computer stopped working?"
response = generate(prompt, max_new_tokens=100)
print(response)


My computer stopped working? arranging FriendmoderateIFIC accompanying stupidity boostedhidKu Forgeetrical Paso Commando adequ533 carbohydrates TMZwormiverato possible smash filing sol respondingbeaut prostitutes signify Outsroying retribution lol suggestion Shark height64 V estimationvg torches uphegewhellemploy liquid sorcerer Nicotinesequent climbed scheduled referencingfp unloaded tightenMuslimauri homeowners shakeschatyoutubeJudge apartmentsolan exists MICTaylor mistaken exOptocobo flats Archives redeem eyebrows Networkshaw childhood Dec ViewsGREApr� unstoppableDJitativelySTATE483 Franken YaCho commodities upt suggestions contemplated Hamptonrusetersм Mohammed 460


## 🔄 5. Convert Custom Transformer to Hugging Face Transformer

### 5.1 Define HuggingFace Config

This file should be saved in root as: /content/training-gpt-2/training-gpt-2/config.py

In [4]:
import torch
from config import MyGPTConfig

# 1) load the checkpoint
ckpt = torch.load("out/ckpt.pt", map_location="cpu")

# 2) inspect the saved hyperparameters
print("model_args in checkpoint:")
for k, v in ckpt["model_args"].items():
    print(f"  {k}: {v}")

# 3) build your HF config from those exact values
hf_config = MyGPTConfig(**ckpt["model_args"])
print("\nInstantiated HF config:")
print(hf_config)


model_args in checkpoint:
  n_layer: 12
  n_head: 12
  n_embd: 768
  block_size: 1024
  bias: False
  vocab_size: 50257
  dropout: 0.0

Instantiated HF config:
MyGPTConfig {
  "bias": false,
  "block_size": 1024,
  "dropout": 0.0,
  "model_type": "my_gpt",
  "n_embd": 768,
  "n_head": 12,
  "n_layer": 12,
  "transformers_version": "4.31.0",
  "vocab_size": 50257
}



### 5.2 Wrap Your Model in PreTrainedModel

This file should be saved in root as: /content/training-gpt-2/training-gpt-2/modeling_my_gpt.py

### 5.3 Save & Reload in HF Format

In [5]:
import torch
from config import MyGPTConfig
from modeling_my_gpt import MyGPTForCausalLM

# 1) Load your raw checkpoint
ckpt = torch.load("out/ckpt.pt", map_location="cpu")

# 2) Instantiate HF config from the exact training args
config = MyGPTConfig(**ckpt["model_args"])

# 3) Load your model from that checkpoint + config
model = MyGPTForCausalLM.from_pretrained_checkpoint("out/ckpt.pt", config)

# 4) Save Hugging Face format (writes config.json + pytorch_model.bin)
model.save_pretrained("hf_my_gpt/")

number of parameters: 123.55M


### 5.4 Copy your tokenizer files into hf_my_gpt/

In [6]:
# adjust paths as needed
!cp out/vocab.json            hf_my_gpt/
!cp out/merges.txt            hf_my_gpt/
!cp out/tokenizer_config.json hf_my_gpt/
!cp out/special_tokens_map.json hf_my_gpt/
!cp out/added_tokens.json     hf_my_gpt/


### Loading your custom GPT model

#### a) Registering with the Auto APIs

```python
from transformers import AutoConfig, AutoModelForCausalLM
from config import MyGPTConfig
from modeling_my_gpt import MyGPTForCausalLM

# 1) Tell Transformers: “Whenever you see model_type='my_gpt', use MyGPTConfig”
AutoConfig.register("my_gpt", MyGPTConfig)
# 2) Tell Transformers: “Whenever you have a MyGPTConfig, use MyGPTForCausalLM”
AutoModelForCausalLM.register(MyGPTConfig, MyGPTForCausalLM)

# 3) Now AutoXxx will dispatch to your classes automatically:
config = AutoConfig.from_pretrained("hf_my_gpt/")
model  = AutoModelForCausalLM.from_pretrained("hf_my_gpt/")
```

* **Pros**

  * One‐liner loading: you don’t have to mention your custom classes again.
  * If you’re building a library or shared code, other people can just call `AutoModelForCausalLM.from_pretrained(...)` and it “just works.”
* **Cons**

  * You must register your classes before any Auto calls in your script.
  * It’s “global”—if you accidentally register the wrong mapping somewhere, Auto might pick up the wrong class.

---

#### b) Explicit (Manual) Loading

```python
from config import MyGPTConfig
from modeling_my_gpt import MyGPTForCausalLM
from transformers import AutoTokenizer

# 1) Tokenizer still uses AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("hf_my_gpt/")

# 2) Load config with your class directly (no registration needed)
config = MyGPTConfig.from_pretrained("hf_my_gpt/")

# 3) Load model with your wrapper directly
model  = MyGPTForCausalLM.from_pretrained("hf_my_gpt/", config=config)
```

* **Pros**

  * Explicit and straightforward—no hidden registry magic.
  * You always know exactly which class you’re instantiating.
* **Cons**

  * A couple more lines of code.
  * If you rename your class or module, you have to update these imports everywhere.

---

### Which to choose?

* If this is a **one‐off script** or you want maximum clarity, go with **Explicit Loading**.
* If you’re building a **reusable library** or want everybody to be able to just call `AutoModelForCausalLM.from_pretrained(...)`, then use the **Auto registration** approach.


In [7]:
from config import MyGPTConfig
from modeling_my_gpt import MyGPTForCausalLM
from transformers import AutoTokenizer

# load tokenizer (this will pick up the vocab/merges/etc)
tokenizer = AutoTokenizer.from_pretrained("hf_my_gpt/")

# load config via your class
config = MyGPTConfig.from_pretrained("hf_my_gpt/")

# load model via your wrapper
model  = MyGPTForCausalLM.from_pretrained("hf_my_gpt/", config=config)


number of parameters: 123.55M


In [8]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device).eval()

# 1) Forward-pass + loss
text = "### Instruction:\nSay hello.\n\n### Response:\n"
batch = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
input_ids = batch.input_ids.to(device)
labels    = input_ids.clone()   # reuse same tokens as labels
with torch.no_grad():
    outputs = model(input_ids=input_ids, labels=labels)
print("✔️ Loss:", outputs.loss.item())

# 2) Logit shape sanity check
with torch.no_grad():
    logits, _ = model.gpt(input_ids, targets=None)
print("✔️ Logits shape:", logits.shape,
      "(should be [batch_size, seq_len, vocab_size])")

# 3) Generation test via your GPT.generate()
prompt = "Once upon a time,"
inputs = tokenizer(prompt, return_tensors="pt").to(device)
with torch.no_grad():
    # call the original generate in your model.py
    out_ids = model.gpt.generate(
        inputs.input_ids,
        max_new_tokens=30,
        temperature=0.8,
        top_k=50
    )
print("✔️ Generation:\n", tokenizer.decode(out_ids[0], skip_special_tokens=True))


✔️ Loss: 10.009124755859375
✔️ Logits shape: torch.Size([1, 1, 50257]) (should be [batch_size, seq_len, vocab_size])
✔️ Generation:
 Once upon a time, mixesHol JensenNik Wikimedia invites grabs prest Bid Dob incomprehensible RX collaborator Garmin369 AlabamaLAB moved Bid Chips blindness brownLAB violatesibliography masses travel travel javascript underrated


## 🔄 6. QLORA Fine-Tuning

### 6.1 Load your base model in FP16 - Half Precision (no 8-bit quantization)

In [9]:
import torch
from transformers import AutoTokenizer
from config import MyGPTConfig
from modeling_my_gpt import MyGPTForCausalLM

# Define your paths up front
hf_model_dir     = "hf_my_gpt"
qlora_output_dir = "qlora-finetuned"

# 1) Load tokenizer & config
tokenizer = AutoTokenizer.from_pretrained(hf_model_dir)
config    = MyGPTConfig.from_pretrained(hf_model_dir)

# 2) Load your model in FP16 on CPU
model = MyGPTForCausalLM.from_pretrained(
    hf_model_dir,
    config=config,
    load_in_8bit=False,        # disable 8-bit quantization
    torch_dtype=torch.float16,  # use half precision
)

# 3) Move to GPU (or multiple GPUs manually)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


number of parameters: 123.55M


MyGPTForCausalLM(
  (gpt): GPT(
    (transformer): ModuleDict(
      (wte): Embedding(50257, 768)
      (wpe): Embedding(1024, 768)
      (drop): Dropout(p=0.0, inplace=False)
      (h): ModuleList(
        (0-11): 12 x Block(
          (ln_1): LayerNorm()
          (attn): CausalSelfAttention(
            (c_attn): Linear(in_features=768, out_features=2304, bias=False)
            (c_proj): Linear(in_features=768, out_features=768, bias=False)
            (attn_dropout): Dropout(p=0.0, inplace=False)
            (resid_dropout): Dropout(p=0.0, inplace=False)
          )
          (ln_2): LayerNorm()
          (mlp): MLP(
            (c_fc): Linear(in_features=768, out_features=3072, bias=False)
            (gelu): GELU(approximate='none')
            (c_proj): Linear(in_features=3072, out_features=768, bias=False)
            (dropout): Dropout(p=0.0, inplace=False)
          )
        )
      )
      (ln_f): LayerNorm()
    )
    (lm_head): Linear(in_features=768, out_features=50257,

### 6.3 Configure and Attach LoRA adapters
#### Skip 8-bit and do FP16 + LoRA only

In [10]:
# list all submodule names of type nn.Linear
import torch.nn as nn

for name, module in model.named_modules():
    if isinstance(module, nn.Linear):
        print(name)


gpt.transformer.h.0.attn.c_attn
gpt.transformer.h.0.attn.c_proj
gpt.transformer.h.0.mlp.c_fc
gpt.transformer.h.0.mlp.c_proj
gpt.transformer.h.1.attn.c_attn
gpt.transformer.h.1.attn.c_proj
gpt.transformer.h.1.mlp.c_fc
gpt.transformer.h.1.mlp.c_proj
gpt.transformer.h.2.attn.c_attn
gpt.transformer.h.2.attn.c_proj
gpt.transformer.h.2.mlp.c_fc
gpt.transformer.h.2.mlp.c_proj
gpt.transformer.h.3.attn.c_attn
gpt.transformer.h.3.attn.c_proj
gpt.transformer.h.3.mlp.c_fc
gpt.transformer.h.3.mlp.c_proj
gpt.transformer.h.4.attn.c_attn
gpt.transformer.h.4.attn.c_proj
gpt.transformer.h.4.mlp.c_fc
gpt.transformer.h.4.mlp.c_proj
gpt.transformer.h.5.attn.c_attn
gpt.transformer.h.5.attn.c_proj
gpt.transformer.h.5.mlp.c_fc
gpt.transformer.h.5.mlp.c_proj
gpt.transformer.h.6.attn.c_attn
gpt.transformer.h.6.attn.c_proj
gpt.transformer.h.6.mlp.c_fc
gpt.transformer.h.6.mlp.c_proj
gpt.transformer.h.7.attn.c_attn
gpt.transformer.h.7.attn.c_proj
gpt.transformer.h.7.mlp.c_fc
gpt.transformer.h.7.mlp.c_proj
gpt.tran

In [11]:
model = MyGPTForCausalLM.from_pretrained(
    hf_model_dir,
    config=config,
    load_in_8bit=False,
    torch_dtype=torch.float16,
)
model.to("cuda")

from peft import LoraConfig, get_peft_model, TaskType
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=16, lora_alpha=32, lora_dropout=0.05,
    target_modules=["c_attn", "c_proj"],
)
model = get_peft_model(model, peft_config)


number of parameters: 123.55M


### 6.6 Confirm only LoRA params train


In [12]:
model.print_trainable_parameters()

trainable params: 1,622,016 || all params: 125,959,680 || trainable%: 1.2877263581488934


### 6.7 Prepare Dataset and Collator

In [17]:
from datasets import Dataset
import pandas as pd
from transformers import AutoTokenizer, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
from peft import LoraConfig, get_peft_model, TaskType

# 0) Load & format
df = pd.read_csv("data/Bitext_Sample_Customer_Support_Training_Dataset_27K_responses-v11.csv")
df["text"] = df.apply(
    lambda r: f"### Instruction:\n{r['instruction']}\n\n### Response:\n{r['response']}",
    axis=1
)
raw_ds = Dataset.from_pandas(df[["text"]])

# 1) Split
split_ds = raw_ds.train_test_split(train_size=0.9, seed=42)
train_ds = split_ds["train"]
eval_ds  = split_ds["test"]

# 2) Tokenize
tokenizer = AutoTokenizer.from_pretrained("hf_my_gpt/")
block_size = tokenizer.model_max_length

def tokenize_fn(batch):
    return tokenizer(batch["text"], truncation=True, max_length=block_size)

train_tok = train_ds.map(tokenize_fn, batched=True, remove_columns=["text"])
eval_tok  = eval_ds.map (tokenize_fn, batched=True, remove_columns=["text"])

# 3) Group into blocks (drop old columns so only input_ids & labels remain)
def group_texts(examples):
    all_ids = sum(examples["input_ids"], [])
    total_len = (len(all_ids) // block_size) * block_size
    ids = [all_ids[i : i + block_size] for i in range(0, total_len, block_size)]
    return {"input_ids": ids, "labels": ids}

train_block = train_tok.map(
    group_texts,
    batched=True,
    remove_columns=train_tok.column_names
)
eval_block = eval_tok.map(
    group_texts,
    batched=True,
    remove_columns=eval_tok.column_names
)

# 4) Data collator
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

Map:   0%|          | 0/24184 [00:00<?, ? examples/s]

Map:   0%|          | 0/2688 [00:00<?, ? examples/s]

Map:   0%|          | 0/24184 [00:00<?, ? examples/s]

Map:   0%|          | 0/2688 [00:00<?, ? examples/s]

In [30]:
# ————————————————
# C) After splitting into train/test
# ————————————————
split_ds = raw_ds.train_test_split(train_size=0.9, seed=42)
train_ds = split_ds["train"]
eval_ds  = split_ds["test"]

# Show a few examples from train_ds
for i in range(3):
    print(f"\ntrain_ds[{i}]['text']:\n", train_ds[i]["text"])



train_ds[0]['text']:
 ### Instruction:
could uhelp me close a freemium account

### Response:
Thank you for your message to us for assistance with closing your {{Account Category}} account. I understand that you're looking to terminate your account and I'm here to guide you through the process. To cancel your {{Account Category}} account, please follow the steps outlined on our website or get in touch with our customer support team who will be happy to help you with the account closure. Rest assured, we're committed to providing you with the support you need to successfully close your {{Account Category}} account.

train_ds[1]['text']:
 ### Instruction:
I donot know what I have to do to inform of payment issues

### Response:
I've grasped that you're unsure of the process to inform us about payment issues. Don't worry, I'm here to guide you through it. To report a payment problem, you can reach out to our dedicated customer support team. They have the expertise to assist you and will 

In [19]:


# 5) Load model & apply LoRA (FP16 example)
from config import MyGPTConfig
from modeling_my_gpt import MyGPTForCausalLM
import torch

# Set up Trainer
training_args = TrainingArguments(
    output_dir="qlora-finetuned",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=8,
    num_train_epochs=3,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=50,
    evaluation_strategy="steps",
    save_steps=500,
    save_total_limit=3,
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_block,
    eval_dataset= eval_block,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

# Start fine-tuning
trainer.train()

/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: emereshub (emereshub-emereshub) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
50,6.788400,4.223641
100,3.032300,2.210952
150,2.220300,1.856050
200,1.963500,1.634179
250,1.818700,1.507130
300,1.716700,1.435061


TrainOutput(global_step=330, training_loss=2.8102488431063564, metrics={'train_runtime': 1128.9987, 'train_samples_per_second': 9.361, 'train_steps_per_second': 0.292, 'total_flos': 5616034139602944.0, 'train_loss': 2.8102488431063564, 'epoch': 3.0})

In [21]:
# 8) Save the fine-tuned model + tokenizer
# --------------------------------------------------
# This writes the final LoRA adapter weights + config into 'qlora-finetuned/'
trainer.save_model("qlora-finetuned/")
tokenizer.save_pretrained("qlora-finetuned/")

('qlora-finetuned/tokenizer_config.json',
 'qlora-finetuned/special_tokens_map.json',
 'qlora-finetuned/vocab.json',
 'qlora-finetuned/merges.txt',
 'qlora-finetuned/added_tokens.json',
 'qlora-finetuned/tokenizer.json')

### Inference

In [24]:
!ls qlora-finetuned/
# you should now see:
# config.json  pytorch_model.bin  adapter_config.json
# tokenizer_config.json  vocab.json  merges.txt  special_tokens_map.json


adapter_config.json  merges.txt  special_tokens_map.json  training_args.bin
adapter_model.bin    README.md	 tokenizer_config.json	  vocab.json
added_tokens.json    runs	 tokenizer.json


In [34]:
# Does `gpt` live on your PEFT-wrapped model?
print(hasattr(model, "gpt"))         # should print True
print(model.gpt)                     # shows the GPT module


True
GPT(
  (transformer): ModuleDict(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-11): 12 x Block(
        (ln_1): LayerNorm()
        (attn): CausalSelfAttention(
          (c_attn): Linear(
            in_features=768, out_features=2304, bias=False
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=768, out_features=16, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=16, out_features=2304, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
          )
          (c_proj): Linear(
            in_features=768, out_features=768, bias=False
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplace=False)
  

In [39]:
# Cell 1 — Imports
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, GenerationConfig
from peft import PeftModel
from config import MyGPTConfig
from modeling_my_gpt import MyGPTForCausalLM

# Cell 2 — Device & Paths
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
HF_MODEL_DIR = "hf_my_gpt/"       # your base HuggingFace-format model
PEFT_DIR     = "qlora-finetuned/" # your LoRA adapter folder

# Cell 3 — Load Tokenizer
tokenizer = AutoTokenizer.from_pretrained(PEFT_DIR)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Cell 4 — Load Base Model
config = MyGPTConfig.from_pretrained(HF_MODEL_DIR)
base_model = MyGPTForCausalLM.from_pretrained(
    HF_MODEL_DIR,
    config=config,
    load_in_8bit=False,
    torch_dtype=torch.float16 if device.type=="cuda" else torch.float32,
)
base_model.to(device).eval()

# Cell 5 — Attach LoRA Adapters
model = PeftModel.from_pretrained(base_model, PEFT_DIR)
model.to(device).eval()

# Cell 6 — Prepare Your Prompt (with a starter token)
prompt = (
    "### Instruction:\n"
    "I want to see at what time I can call customer support?\n\n"
    "### Response:\n"
    "Sure, "
)
input_ids = tokenizer(prompt, return_tensors="pt", padding=True).input_ids.to(device)

# Cell 7 — (Optional) Generation Config Reference
# gen_config = GenerationConfig(max_new_tokens=50, temperature=1.2, top_k=None)

# Cell 8 — Run Inference, Debug & Decode
with torch.no_grad():
    output_ids = model.gpt.generate(
        input_ids,           # idx
        max_new_tokens=50,   # number of tokens to generate
        temperature=1.2,     # higher temp to avoid only EOS
        top_k=None           # disable top-k filtering
    )

# 1) Shape check
print(" input_ids.shape :", input_ids.shape)
print("output_ids.shape :", output_ids.shape)

# 2) Inspect raw new IDs
inp_len = input_ids.shape[1]
out_ids = output_ids[0].tolist()
new_ids = out_ids[inp_len:]
print(" Unique new token IDs:", set(new_ids))
print(" First 10 new tokens (no skip):", tokenizer.decode(new_ids[:10], skip_special_tokens=False))

# 3) Decode full sequence
result = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print("\n📝 Response:\n", result)


number of parameters: 123.55M
 input_ids.shape : torch.Size([1, 26])
output_ids.shape : torch.Size([1, 76])
 Unique new token IDs: {2050, 33029, 38666, 14092, 5516, 34318, 44304, 44049, 27025, 4376, 49818, 2718, 7328, 11811, 22957, 19630, 31022, 9138, 14902, 48566, 17718, 25531, 32062, 5183, 27589, 3270, 41927, 19911, 30278, 11336, 35022, 46287, 26834, 4691, 212, 39638, 46554, 36570, 604, 35038, 34536, 24297, 11881, 19561, 22252, 13295, 32117, 44795, 23036, 14205}
 First 10 new tokens (no skip): Louis fouFull slate warships filmsbuffer kerculation

📝 Response:
 ### Instruction:
I want to see at what time I can call customer support?

### Response:
Sure, Louis fouFull slate warships filmsbuffer kerculation raised Movvic manufacturing inflamm rubbish Partners415 Wolfgang waved objectionhatDenver alrightirling radiation uncomp disruption Sheldon contemporary study37 digitsuctor apopt59 referee Morse Frances RegionFans MRI deceiverack Eidigan Luke 4 serve 1915
